In [1]:
# Notebook: embeddings_generation.ipynb

import os
import json
import pathlib
import logging
from typing import List, Dict, Any
import numpy as np

# Configuración de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Intentar importar las bibliotecas necesarias
try:
    from sentence_transformers import SentenceTransformer
    SENTENCE_TRANSFORMERS_AVAILABLE = True
except ImportError:
    SENTENCE_TRANSFORMERS_AVAILABLE = False
    logger.warning("sentence-transformers no está instalado. Necesario para generar embeddings.")
    
try:
    import chromadb
    CHROMADB_AVAILABLE = True
except ImportError:
    CHROMADB_AVAILABLE = False
    logger.warning("chromadb no está instalado. Necesario para la base de datos vectorial.")


def load_chunks(chunks_path: str) -> List[Dict[str, Any]]:
    """
    Carga los chunks procesados desde un archivo JSON.
    
    Args:
        chunks_path: Ruta al archivo JSON con todos los chunks
        
    Returns:
        Lista de diccionarios con los chunks y sus metadatos
    """
    logger.info(f"Cargando chunks desde {chunks_path}")
    
    try:
        with open(chunks_path, 'r', encoding='utf-8') as f:
            chunks = json.load(f)
        logger.info(f"Cargados {len(chunks)} chunks")
        return chunks
    except Exception as e:
        logger.error(f"Error al cargar chunks: {e}")
        return []


def generate_embeddings(chunks: List[Dict[str, Any]], model_name: str = "all-MiniLM-L6-v2") -> Dict[str, Any]:
    """
    Genera embeddings para cada chunk de texto utilizando un modelo de sentence-transformers.
    
    Args:
        chunks: Lista de diccionarios con los chunks y sus metadatos
        model_name: Nombre del modelo de sentence-transformers a utilizar
        
    Returns:
        Diccionario con textos, embeddings y metadatos
    """
    if not SENTENCE_TRANSFORMERS_AVAILABLE:
        raise ImportError("sentence-transformers es necesario para generar embeddings.")
    
    logger.info(f"Cargando modelo {model_name}...")
    model = SentenceTransformer(model_name)
    logger.info("Modelo cargado. Generando embeddings...")
    
    # Extraer textos y metadatos
    texts = [chunk["text"] for chunk in chunks]
    metadatas = [chunk["metadata"] for chunk in chunks]
    
    # Generar embeddings (usando GPU si está disponible)
    embeddings = model.encode(texts, show_progress_bar=True, batch_size=32)
    
    logger.info(f"Generados {len(embeddings)} embeddings de dimensión {embeddings.shape[1]}")
    
    return {
        "texts": texts,
        "embeddings": embeddings,
        "metadatas": metadatas
    }


def save_embeddings(embeddings_data: Dict[str, Any], output_dir: str, model_name: str) -> None:
    """
    Guarda los embeddings generados en archivos.
    
    Args:
        embeddings_data: Diccionario con textos, embeddings y metadatos
        output_dir: Directorio donde guardar los embeddings
        model_name: Nombre del modelo utilizado (para nombrar los archivos)
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Guardar embeddings como numpy array
    embeddings_path = os.path.join(output_dir, f"{model_name.replace('/', '_')}_embeddings.npy")
    np.save(embeddings_path, embeddings_data["embeddings"])
    logger.info(f"Embeddings guardados en {embeddings_path}")
    
    # Guardar textos y metadatos en JSON
    metadata_path = os.path.join(output_dir, f"{model_name.replace('/', '_')}_metadata.json")
    with open(metadata_path, 'w', encoding='utf-8') as f:
        json.dump({
            "texts": embeddings_data["texts"],
            "metadatas": embeddings_data["metadatas"]
        }, f, ensure_ascii=False, indent=2)
    logger.info(f"Metadatos y textos guardados en {metadata_path}")


def create_vector_database(embeddings_data: Dict[str, Any], db_path: str, collection_name: str = "bibliography") -> None:
    """
    Crea una base de datos vectorial con Chroma.
    
    Args:
        embeddings_data: Diccionario con textos, embeddings y metadatos
        db_path: Ruta donde guardar la base de datos
        collection_name: Nombre de la colección en la base de datos
    """
    if not CHROMADB_AVAILABLE:
        raise ImportError("chromadb es necesario para crear la base de datos vectorial.")
    
    logger.info(f"Creando base de datos vectorial en {db_path}")
    
    # Crear cliente y colección persistente
    client = chromadb.PersistentClient(path=db_path)
    
    # Intentar obtener la colección existente o crear una nueva
    try:
        collection = client.get_collection(name=collection_name)
        logger.info(f"Colección existente '{collection_name}' encontrada. Se añadirán/actualizarán documentos.")
    except Exception:
        collection = client.create_collection(name=collection_name)
        logger.info(f"Creada nueva colección '{collection_name}'")
    
    # Preparar IDs para los documentos
    ids = [f"chunk_{i}" for i in range(len(embeddings_data["texts"]))]
    
    # Añadir documentos a la colección
    collection.add(
        embeddings=embeddings_data["embeddings"].tolist(),
        documents=embeddings_data["texts"],
        metadatas=embeddings_data["metadatas"],
        ids=ids
    )
    
    logger.info(f"Añadidos {len(ids)} documentos a la colección '{collection_name}'")


# Definir rutas
current_dir = pathlib.Path(os.getcwd())
chunks_path = os.path.join(current_dir, "chunks", "processed", "all_chunks.json")
embeddings_dir = os.path.join(current_dir.parent, "models", "embeddings")
db_dir = os.path.join(current_dir.parent, "models", "indexes", "chroma_db")

# Configuración del modelo de embeddings
model_name = "all-MiniLM-L6-v2"  # Modelo pequeño y rápido, pero puedes usar otros como "paraphrase-multilingual-MiniLM-L12-v2" para mejor soporte multilingüe

# Imprimir las rutas para verificar
print(f"Archivo de chunks: {chunks_path}")
print(f"Directorio de embeddings: {embeddings_dir}")
print(f"Directorio de base de datos: {db_dir}")
print(f"Modelo a utilizar: {model_name}")

# Preguntar al usuario si las rutas son correctas antes de continuar
confirmation = input("¿Son correctas estas rutas y configuración? (s/n): ")
if confirmation.lower() != 's':
    print("Proceso cancelado.")
else:
    # Cargar chunks
    chunks = load_chunks(chunks_path)
    
    if not chunks:
        print("No se pudieron cargar los chunks. Verifica la ruta y el formato del archivo.")
    else:
        # Generar embeddings
        embeddings_data = generate_embeddings(chunks, model_name)
        
        # Guardar embeddings
        save_embeddings(embeddings_data, embeddings_dir, model_name)
        
        # Crear base de datos vectorial
        create_vector_database(embeddings_data, db_dir, "bibliography")
        
        print("Proceso completado exitosamente.")

c:\Users\Usuario\anaconda3\envs\dependencias\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-01 11:54:22,410 - INFO - NumExpr defaulting to 16 threads.


Archivo de chunks: c:\Users\Usuario\Documents\Github\Seguridad económica\src\rag\extraction\chunks\processed\all_chunks.json
Directorio de embeddings: c:\Users\Usuario\Documents\Github\Seguridad económica\src\rag\models\embeddings
Directorio de base de datos: c:\Users\Usuario\Documents\Github\Seguridad económica\src\rag\models\indexes\chroma_db
Modelo a utilizar: all-MiniLM-L6-v2


2025-04-01 11:54:32,331 - INFO - Cargando chunks desde c:\Users\Usuario\Documents\Github\Seguridad económica\src\rag\extraction\chunks\processed\all_chunks.json
2025-04-01 11:54:32,400 - INFO - Cargados 6625 chunks
2025-04-01 11:54:32,400 - INFO - Cargando modelo all-MiniLM-L6-v2...
2025-04-01 11:54:32,428 - INFO - Use pytorch device_name: cuda
2025-04-01 11:54:32,428 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
c:\Users\Usuario\anaconda3\envs\dependencias\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https

Proceso completado exitosamente.
